## Hyperparameter tuning using RandomSearchCV

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from keras.datasets import mnist
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras import Model, Sequential, models, layers

np.random.seed(6)

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = 28, 28
num_classes = 10

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [10]:
max_int = np.random.randint(5,6)
min_int = np.random.randint(2,4)

hidden_layers = []

for rep in range(min_int, max_int):
    layer = np.repeat(np.random.randint(32, 64),rep).tolist()
    hidden_layers.append(layer)

dropout_rates = np.round(np.arange(0.1,0.5,0.1),1).tolist()
epochs = list(range(1,10))
batches = list(range(50,200))
# l2_penalty_opts = np.arange(0.1,0.5,0.1).tolist()
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
callbacks = [EarlyStopping(monitor = 'val_loss', patience = 1)]

fit_params = {   
    'callbacks': callbacks,
    'validation_data': (x_test, y_test),
    'verbose': 0,
    'epochs': epochs,
    'batch_size': batches
}

fit_params = dict(callbacks = callbacks,)

param_options = {
    'hidden_layers': hidden_layers,
    'dropout_rate': dropout_rates,
    'optimizer': optimizers
}

In [11]:
hidden_layers

[[41, 41], [35, 35, 35], [52, 52, 52, 52]]

In [14]:
def cnn_model(hidden_layers = [32, 64], activation = 'relu', dropout_rate = 0, optimizer = 'SGD',
                     input_shape = input_shape, num_classes = num_classes):
  
    model = Sequential()
    
    for index, units in enumerate(hidden_layers):
        if index == 0:
            model.add(layers.Conv2D(units, (3,3), input_shape = input_shape, activation = activation))
        else:
            model.add(layers.Conv2D(units, (3,3), activation = activation))

    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    if dropout_rate:
        model.add(layers.Dropout(dropout_rate))

    model.add(layers.Flatten())
    neurons = max(hidden_layers)*2
    model.add(layers.Dense(neurons, activation='relu'))
    
    if dropout_rate:
        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(num_classes, activation = "softmax"))
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])   
    return model

In [18]:
kc_model = KerasClassifier(build_fn = cnn_model)

kc_model_cv = RandomizedSearchCV( 
    kc_model, 
    param_distributions = param_options,
    n_iter = 3,
    cv = 3,
    scoring="neg_log_loss",
#     n_jobs = -1,
    verbose=10
)

kc_model_cv.fit(x_train, y_train)

print('Best score obtained: {0}'.format(kc_model_cv.best_score_))
print('Parameters:')
for param, value in kc_model_cv.best_params_.items():
    print('\t{}: {}'.format(param, value))
    
# Show the results
# print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
# means = random_search.cv_results_['mean_test_score']
# stds = random_search.cv_results_['std_test_score']
# params = random_search.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] optimizer=RMSprop, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 40000 samples
40000/40000 [==============================] - 151s 4ms/sample - loss: 2.8318 - accuracy: 0.7867
[CV]  optimizer=RMSprop, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2, score=-0.081, total= 2.7min
[CV] optimizer=RMSprop, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


Train on 40000 samples
40000/40000 [==============================] - 144s 4ms/sample - loss: 0.2416 - accuracy: 0.9441
[CV]  optimizer=RMSprop, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2, score=-0.102, total= 2.6min
[CV] optimizer=RMSprop, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2 


...

Train on 40000 samples
40000/40000 [==============================] - 139s 3ms/sample - loss: 0.2303 - accuracy: 0.9429
[CV]  optimizer=Nadam, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2, score=-0.063, total= 2.5min
[CV] optimizer=Nadam, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2 


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 16.3min remaining:    0.0s


Train on 40000 samples
40000/40000 [==============================] - 132s 3ms/sample - loss: 0.2618 - accuracy: 0.9404
[CV]  optimizer=Nadam, hidden_layers=[52, 52, 52, 52], dropout_rate=0.2, score=-0.068, total= 2.4min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 18.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 18.6min finished


Train on 60000 samples
60000/60000 [==============================] - 197s 3ms/sample - loss: 0.2116 - accuracy: 0.9542
Best score obtained: -0.06588739765515375
Parameters:
	optimizer: Nadam
	hidden_layers: [52, 52, 52, 52]
	dropout_rate: 0.2
